In [21]:
#import packages

import numpy as np
import pandas as pd
import random
import pickle
import itertools
import folium #mapping
import seaborn as sns #color palette
#from colormap import rgb2hex #chnage RGB codes to hex codes

In [22]:
## USE THIS CODE TO IMPORT DATA

file_name = '../data/results/results_pooled_cement_only_complexcost.pkl' ##results_toydata_uniform.pkl'
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

In [23]:
#convert to dataframe

df = pd.DataFrame(loaded_list, columns = ['supplier', 'receiver', 'distance', 'volume'])
df.head()

#convert suppliers to list to get rid of weird numpy object datatype
supplier_split = df.supplier.to_list()
receiver_split = df.receiver.to_list()
#supplier_split[0:5]


#convert suppliers to lat and lon
supplier_lat= [supplier_split[j][0] for j in range(0,len(supplier_split))]
supplier_lon = [supplier_split[j][1] for j in range(0,len(supplier_split))]

#convert to list of receiver lat and long
receiver_lat= [receiver_split[j][0] for j in range(0,len(receiver_split))]
receiver_lon = [receiver_split[j][1] for j in range(0,len(receiver_split))]

#add back into the df
df['supplier_lat'] = supplier_lat
df['supplier_lon'] = supplier_lon
df['receiver_lat'] = receiver_lat
df['receiver_lon'] = receiver_lat

df.head()

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[34.6222, -117.1001]","[37.68627, -121.674512]",2.000792e+06,8.464567,34.6222,-117.1001,37.686270,37.686270
1,"[34.6222, -117.1001]","[39.06031418, -121.9646343]",2.441070e+06,12.532808,34.6222,-117.1001,39.060314,39.060314
2,"[34.6222, -117.1001]","[39.127373, -121.961772]",2.037133e+06,4.527559,34.6222,-117.1001,39.127373,39.127373
3,"[34.6222, -117.1001]","[39.15124715, -121.933119]",6.272237e+06,5.905512,34.6222,-117.1001,39.151247,39.151247
4,"[34.6222, -117.1001]","[39.033785, -121.91461]",2.024923e+06,14.566929,34.6222,-117.1001,39.033785,39.033785


In [24]:
#create a color palette, repeat it 100 times to see how many things I can visualize at once in the html
palette = ['red', 'green', 'purple', 'orange',  'darkblue',
           'gray','cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
           'black','lightgray','lightred','blue','beige']*100
            #‘lightgreen’] ‘gray’, ‘black’, ‘lightgray’] 'darkred','lightred','blue','beige','white',


#palette

## Map Everything

Shape of dataframe

In [25]:
df.shape

(1841, 8)

In [26]:
#view a record
df[df['supplier_lat'] == 35.399380]
df

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[34.6222, -117.1001]","[37.68627, -121.674512]",2.000792e+06,8.464567,34.6222,-117.1001,37.686270,37.686270
1,"[34.6222, -117.1001]","[39.06031418, -121.9646343]",2.441070e+06,12.532808,34.6222,-117.1001,39.060314,39.060314
2,"[34.6222, -117.1001]","[39.127373, -121.961772]",2.037133e+06,4.527559,34.6222,-117.1001,39.127373,39.127373
3,"[34.6222, -117.1001]","[39.15124715, -121.933119]",6.272237e+06,5.905512,34.6222,-117.1001,39.151247,39.151247
4,"[34.6222, -117.1001]","[39.033785, -121.91461]",2.024923e+06,14.566929,34.6222,-117.1001,39.033785,39.033785
...,...,...,...,...,...,...,...,...
1836,"[34.6045, -117.3382]","[35.418293, -119.699501]",2.001340e+06,32.217848,34.6045,-117.3382,35.418293,35.418293
1837,"[34.6045, -117.3382]","[33.771553999999995, -118.242047]",2.006766e+06,35.425114,34.6045,-117.3382,33.771554,33.771554
1838,"[34.6045, -117.3382]","[34.39482221, -118.49278429999998]",2.003194e+06,34.776903,34.6045,-117.3382,34.394822,34.394822
1839,"[34.6045, -117.3382]","[34.051497, -118.362618]",2.064796e+06,16.505071,34.6045,-117.3382,34.051497,34.051497


Get counts by supplier

In [27]:
#counts by each supplier
df_counts = df.groupby(['supplier_lat','supplier_lon']).count().reset_index()
df_sorted = df_counts.sort_values('distance')
df_sorted.tail(20)

,supplier_lat,supplier_lon,supplier,receiver,distance,volume,receiver_lat,receiver_lon
5,37.318100,-122.091000,7,7,7,7,7,7
1,34.604500,-117.338200,9,9,9,9,9,9
0,34.437557,-116.891034,20,20,20,20,20,20
4,35.029298,-118.316236,30,30,30,30,30,30
3,34.819863,-118.748732,79,79,79,79,79,79
6,40.736900,-122.322300,508,508,508,508,508,508
2,34.622200,-117.100100,1188,1188,1188,1188,1188,1188


In [28]:
#df = df[df['volume'] != 12834.30000]
df

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[34.6222, -117.1001]","[37.68627, -121.674512]",2.000792e+06,8.464567,34.6222,-117.1001,37.686270,37.686270
1,"[34.6222, -117.1001]","[39.06031418, -121.9646343]",2.441070e+06,12.532808,34.6222,-117.1001,39.060314,39.060314
2,"[34.6222, -117.1001]","[39.127373, -121.961772]",2.037133e+06,4.527559,34.6222,-117.1001,39.127373,39.127373
3,"[34.6222, -117.1001]","[39.15124715, -121.933119]",6.272237e+06,5.905512,34.6222,-117.1001,39.151247,39.151247
4,"[34.6222, -117.1001]","[39.033785, -121.91461]",2.024923e+06,14.566929,34.6222,-117.1001,39.033785,39.033785
...,...,...,...,...,...,...,...,...
1836,"[34.6045, -117.3382]","[35.418293, -119.699501]",2.001340e+06,32.217848,34.6045,-117.3382,35.418293,35.418293
1837,"[34.6045, -117.3382]","[33.771553999999995, -118.242047]",2.006766e+06,35.425114,34.6045,-117.3382,33.771554,33.771554
1838,"[34.6045, -117.3382]","[34.39482221, -118.49278429999998]",2.003194e+06,34.776903,34.6045,-117.3382,34.394822,34.394822
1839,"[34.6045, -117.3382]","[34.051497, -118.362618]",2.064796e+06,16.505071,34.6045,-117.3382,34.051497,34.051497


Show the 10 suppliers with the shortest mean distance

In [29]:
#group the dataframe by mean distance
df_grouped = df.groupby(['supplier_lat','supplier_lon']).mean('distance').reset_index()
df_grouped

,supplier_lat,supplier_lon,distance,volume,receiver_lat,receiver_lon
0,34.437557,-116.891034,2.146734e+06,21.085000,34.441330,34.441330
1,34.604500,-117.338200,2.359245e+06,30.055556,35.255004,35.255004
2,34.622200,-117.100100,2.182669e+06,10.803283,35.545885,35.545885
3,34.819863,-118.748732,2.379569e+06,31.351899,36.374201,36.374201
4,35.029298,-118.316236,2.021796e+06,23.733333,35.445581,35.445581
5,37.318100,-122.091000,2.186905e+06,66.414286,33.881513,33.881513
6,40.736900,-122.322300,2.346808e+06,2.110630,35.456693,35.456693


In [30]:
#sort on distance
df_grouped = df_grouped.sort_values('distance')
df_grouped

,supplier_lat,supplier_lon,distance,volume,receiver_lat,receiver_lon
4,35.029298,-118.316236,2.021796e+06,23.733333,35.445581,35.445581
0,34.437557,-116.891034,2.146734e+06,21.085000,34.441330,34.441330
2,34.622200,-117.100100,2.182669e+06,10.803283,35.545885,35.545885
5,37.318100,-122.091000,2.186905e+06,66.414286,33.881513,33.881513
6,40.736900,-122.322300,2.346808e+06,2.110630,35.456693,35.456693
1,34.604500,-117.338200,2.359245e+06,30.055556,35.255004,35.255004
3,34.819863,-118.748732,2.379569e+06,31.351899,36.374201,36.374201


In [31]:
#cut off at the top 10
df_grouped = df_grouped.reset_index() #10 at the .tail(10) head(10)
df_grouped

,index,supplier_lat,supplier_lon,distance,volume,receiver_lat,receiver_lon
0,4,35.029298,-118.316236,2.021796e+06,23.733333,35.445581,35.445581
1,0,34.437557,-116.891034,2.146734e+06,21.085000,34.441330,34.441330
2,2,34.622200,-117.100100,2.182669e+06,10.803283,35.545885,35.545885
3,5,37.318100,-122.091000,2.186905e+06,66.414286,33.881513,33.881513
4,6,40.736900,-122.322300,2.346808e+06,2.110630,35.456693,35.456693
5,1,34.604500,-117.338200,2.359245e+06,30.055556,35.255004,35.255004
6,3,34.819863,-118.748732,2.379569e+06,31.351899,36.374201,36.374201


In [32]:
df_small = pd.DataFrame()

#loop through and get the full dataset. can't group by the lat/long because it's some unhashable type.
for i in range(0,df_grouped.shape[0]):
    this_lat = df_grouped['supplier_lat'][i]
    this_lon = df_grouped['supplier_lon'][i]
    receivers = df[(df['supplier_lat'] == this_lat) & (df['supplier_lon'] == this_lon)]
    df_small = df_small.append(receivers)
    
df_small.shape

(1841, 8)

In [33]:
#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers.shape
#unique_suppliers['supplier_lon'][0]

(7, 2)

In [34]:
vols = df['volume']
sum(vols)

18252.39999259599

In [35]:
#determine the maximum volume - this will be used to scale the line weight
volumes = df['volume']
max_volume = np.max(volumes)
max_volume



173.42954

In [36]:
volumes = df['volume']
quantiles = np.quantile(volumes, [0.2,0.4,0.6,0.8])
quantiles
volumes

0        8.464567
1       12.532808
2        4.527559
3        5.905512
4       14.566929
          ...    
1836    32.217848
1837    35.425114
1838    34.776903
1839    16.505071
1840    15.748293
Name: volume, Length: 1841, dtype: float64

In [38]:
#create map

#center the map in Cali
#mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=20 )
receiver_count = 0
 
#plot all the suppliers 
latlon = df_small['supplier']
mapit = folium.Map( location=[37.3427, -119.2244], zoom_start=6)

#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers

#loop through suppliers, paint all the receivers in the same color
for i in range(0,unique_suppliers.shape[0]):
    
    #big circle for supplier
    folium.CircleMarker( location=[ unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i] ], fill=True, 
                        fill_color=palette[i], color=palette[i],  radius=10 ).add_to( mapit ) #'
    
    #find all receivers associated with the supplier late and lon, filter the df
    this_lat = unique_suppliers['supplier_lat'][i]
    this_lon = unique_suppliers['supplier_lon'][i]
    receivers = df_small[(df_small['supplier_lat'] == this_lat) & (df_small['supplier_lon'] == this_lon)]
    
    #plot the receivers with a new color each time
    latlon = receivers['receiver']
    #print(receivers.shape[0])
    #print(receivers.shape[1])
    receiver_count += receivers.shape[0] #verify 1000
    
    #for each receiver  
    for j in range(0,len(latlon)): #coord in latlon: 
        
        lat = latlon.iloc[j][0]
        lon = latlon.iloc[j][1]
        
        volume = receivers.iloc[j]['volume']
        #if volume > quantiles[3]:
        #    quantile = 1 #80-100 percentile
        #elif volume > quantiles[2]:
        #    quantile = 0.8 #60-80 percentile
        #elif volume > quantiles[1]:
        #    quantile = 0.6 #40-60 percentile
        #elif volume > quantiles[0]:
        #    quantile = 0.4 #20-40 percentile
        #else:
        #    quantile  = 0.2 #0-20 percentile
        #line_weight = quantile * 5
        line_weight = volume/max_volume * 20

        #print(line_weight)
        #print(volume, quantile)
        
        #line coordinates
        line_coords = [ [unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i]],
                      [lat, lon] ]#[coord[0], coord[1] ]  ]
        my_PolyLine= folium.PolyLine(locations=line_coords,color=palette[i],weight=line_weight) #color and supplier use i index
        mapit.add_child(my_PolyLine)
        
        #dot for the receiver
        folium.CircleMarker( location=[ lat,lon ], fill=True, fill_color=palette[i], color=palette[i],  radius=1 ).add_to( mapit )

print(receiver_count)
mapit

mapit.save(outfile= "visualization.html")

1841
